In [1]:
import requests
import pandas as pd
import re
from bs4 import BeautifulSoup
from datetime import date

In [2]:
df_old = pd.read_csv("brd_covid_19.csv",parse_dates=[5])

In [3]:
df_old.sort_values(["bundesland","datum"], ascending = False).head(20)

,bundesland,faelle,tod,lat,long,datum
237,Thüringen,46,0,50.85,11.13,2020-03-15
220,Thüringen,29,0,50.85,11.13,2020-03-14
32,Thüringen,14,0,50.85,11.13,2020-03-13
15,Thüringen,10,0,50.85,11.13,2020-03-12
203,Thüringen,10,0,50.85,11.13,2020-03-11
186,Thüringen,2,0,50.85,11.13,2020-03-10
170,Thüringen,2,0,50.85,11.13,2020-03-09
154,Thüringen,2,0,50.85,11.13,2020-03-08
138,Thüringen,1,0,50.85,11.13,2020-03-07
122,Thüringen,1,0,50.85,11.13,2020-03-06


In [4]:
today = pd.Timestamp(date.today())

In [5]:
URL = "https://www.rki.de/DE/Content/InfAZ/N/Neuartiges_Coronavirus/Fallzahlen.html"

In [6]:
page = requests.get(URL)

In [7]:
soup = BeautifulSoup(page.content, 'html.parser')

In [8]:
results = soup.find_all("tr")

In [9]:
bundesland = []
faelle = []
tod = []
for idx,elm in enumerate(results):
    if idx > 0:
        bundesland.append(re.sub('<[^<]+?>', '', str(elm.find_all("td")[0])))
        faelle.append(re.sub('<[^<]+?>', '', str(elm.find_all("td")[1])).split()[0])
        if len(re.sub('<[^<]+?>', '', str(elm.find_all("td")[1])).split()) > 1:
            tod.append(re.sub('<[^<]+?>', '', str(elm.find_all("td")[1])).split()[1].strip("(").strip(")"))
        else:
            tod.append("0")
    else:
        pass

In [10]:
lat={"Baden-Württemberg":48.5, "Bayern":48.9, "Berlin":52.5, "Brandenburg":52, "Bremen":53.1, "Hamburg":53.5, 
     "Hessen":50.7, "Mecklenburg-Vorpommern":53.8, "Niedersachsen":52.57, "Nordrhein-Westfalen":51.6,"Rheinland-Pfalz":50, 
     "Saarland":49.4, "Sachsen":51.05, "Sachsen-Anhalt":52, "Schleswig-Holstein":54.2, "Thüringen":50.85, "Gesamt":51.06}
long={"Baden-Württemberg":9, "Bayern":11.4, "Berlin":13.4, "Brandenburg":13.87, "Bremen":8.8, "Hamburg":10, 
      "Hessen":9, "Mecklenburg-Vorpommern":12.5,"Niedersachsen":10, "Nordrhein-Westfalen":7.5,"Rheinland-Pfalz":7.3,
      "Saarland":6.97,"Sachsen":13.5, "Sachsen-Anhalt":11.65,"Schleswig-Holstein":9.9, "Thüringen":11.13,"Gesamt":10}


In [11]:
df=pd.DataFrame(data=[bundesland,faelle,tod]).T
    

In [12]:
df["lat"]="NaN"
df["long"]="NaN"
df["date"] = today

In [13]:
df.columns = ["bundesland","faelle","tod","lat","long","datum"]

In [14]:
df.tod = df.tod.replace(to_replace = "", value = "0", regex = True)

In [15]:
df.datum = pd.to_datetime(df.datum)

In [16]:
df.bundesland.replace("Schleswig Holstein","Schleswig-Holstein", inplace=True)
lat2=[]
long2=[]
for elm in df.bundesland.values:
    lat2.append(lat[elm])
    long2.append(long[elm])

In [17]:
df["lat"] = lat2
df["long"] = long2

In [18]:
for idx,elm in enumerate(df.faelle):
    df.faelle[idx] = elm.replace(".","")

for idx,elm in enumerate(df.tod):
    df.tod[idx] = elm.replace(".","")

/Users/karstenyan/opt/anaconda3/envs/nf/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/karstenyan/opt/anaconda3/envs/nf/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 6 columns):
bundesland    17 non-null object
faelle        17 non-null object
tod           17 non-null object
lat           17 non-null float64
long          17 non-null float64
datum         17 non-null datetime64[ns]
dtypes: datetime64[ns](1), float64(2), object(3)
memory usage: 896.0+ bytes


In [20]:
#df.lat = df.lat.astype("float")
#df.long = df.long.astype("float")
df.tod = df.tod.astype("int64")
df.faelle = df.faelle.astype("int64")

In [21]:
df=pd.concat([df_old,df]).reset_index().drop("index",axis=1)

In [22]:
df[df["datum"]==today]

,bundesland,faelle,tod,lat,long,datum
239,Baden-Württemberg,827,3,48.50,9.00,2020-03-16
240,Bayern,886,4,48.90,11.40,2020-03-16
241,Berlin,265,0,52.50,13.40,2020-03-16
242,Brandenburg,84,0,52.00,13.87,2020-03-16
243,Bremen,53,0,53.10,8.80,2020-03-16
244,Hamburg,162,0,53.50,10.00,2020-03-16
245,Hessen,286,0,50.70,9.00,2020-03-16
246,Mecklenburg-Vorpommern,50,0,53.80,12.50,2020-03-16
247,Niedersachsen,287,0,52.57,10.00,2020-03-16
248,Nordrhein-Westfalen,1407,5,51.60,7.50,2020-03-16


In [23]:
df_old.to_csv ('old_data/brd_covid_19'+str(today)+".csv", index = False, header=True)

In [24]:
df.to_csv ('brd_covid_19.csv', index = False, header=True)